In [1]:
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt
from math import sqrt

#estimators
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

#model metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#cross validation
from sklearn.model_selection import train_test_split


In [2]:
from feature_engine.discretisers import EqualWidthDiscretiser

In [3]:
data = pd.read_csv("Credit_One_DataSet.csv")

In [4]:
data.head()

,LIMIT_BAL,DPNM,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,1,2,2,1,24,2,2,-1,-1,...,689,0,0,0,0,689,0,0,0,0
1,120000,1,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,0,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,0,2,2,1,37,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,0,1,2,1,57,-1,0,-1,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


In [5]:
#data = data[data['DPNM']==1]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   LIMIT_BAL  30000 non-null  int64
 1   DPNM       30000 non-null  int64
 2   SEX        30000 non-null  int64
 3   EDUCATION  30000 non-null  int64
 4   MARRIAGE   30000 non-null  int64
 5   AGE        30000 non-null  int64
 6   PAY_0      30000 non-null  int64
 7   PAY_2      30000 non-null  int64
 8   PAY_3      30000 non-null  int64
 9   PAY_4      30000 non-null  int64
 10  PAY_5      30000 non-null  int64
 11  PAY_6      30000 non-null  int64
 12  BILL_AMT1  30000 non-null  int64
 13  BILL_AMT2  30000 non-null  int64
 14  BILL_AMT3  30000 non-null  int64
 15  BILL_AMT4  30000 non-null  int64
 16  BILL_AMT5  30000 non-null  int64
 17  BILL_AMT6  30000 non-null  int64
 18  PAY_AMT1   30000 non-null  int64
 19  PAY_AMT2   30000 non-null  int64
 20  PAY_AMT3   30000 non-null  int64
 21  PAY_AMT4   3

In [6]:
SEED=123

In [7]:
data = data.drop(['SEX','MARRIAGE','DPNM'], axis=1)
data = data.sample(1000, random_state=SEED) #Obtengo el mismo resultado con 30000

In [8]:
data.head()

,LIMIT_BAL,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
25665,40000,2,26,-1,0,0,0,-1,0,28991,...,13600,0,44024,18697,1300,1000,0,22373,680,10000
16464,80000,3,59,0,0,0,0,0,0,75662,...,53169,50875,50372,49470,3212,2106,2000,1603,1903,2006
22386,170000,1,30,2,2,2,2,2,2,155910,...,161487,157577,168094,170922,6800,6500,0,13000,5500,1000
10149,200000,2,41,-2,-2,-2,-2,-2,-2,23570,...,0,0,0,0,742,0,0,0,0,0
8729,50000,2,43,0,0,0,0,0,0,8214,...,9044,9225,9417,9617,1140,1150,331,341,356,330


In [9]:
data['BILL_AMT_total']=data['BILL_AMT1']+data['BILL_AMT2']+data['BILL_AMT3']+data['BILL_AMT4']+data['BILL_AMT5']+data['BILL_AMT6']
data['PAY_AMT_total']=data['PAY_AMT1']+data['PAY_AMT2']+data['PAY_AMT3']+data['PAY_AMT4']+data['PAY_AMT5']+data['PAY_AMT6']
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 25665 to 10093
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   LIMIT_BAL       1000 non-null   int64
 1   EDUCATION       1000 non-null   int64
 2   AGE             1000 non-null   int64
 3   PAY_0           1000 non-null   int64
 4   PAY_2           1000 non-null   int64
 5   PAY_3           1000 non-null   int64
 6   PAY_4           1000 non-null   int64
 7   PAY_5           1000 non-null   int64
 8   PAY_6           1000 non-null   int64
 9   BILL_AMT1       1000 non-null   int64
 10  BILL_AMT2       1000 non-null   int64
 11  BILL_AMT3       1000 non-null   int64
 12  BILL_AMT4       1000 non-null   int64
 13  BILL_AMT5       1000 non-null   int64
 14  BILL_AMT6       1000 non-null   int64
 15  PAY_AMT1        1000 non-null   int64
 16  PAY_AMT2        1000 non-null   int64
 17  PAY_AMT3        1000 non-null   int64
 18  PAY_AMT4        1000 no

In [10]:
#X = data.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
X = data.iloc[:,[2,3,5,6,21,22]]

In [11]:
print("Features sample")
X.head()

Features sample


,AGE,PAY_0,PAY_3,PAY_4,BILL_AMT_total,PAY_AMT_total
25665,26,-1,0,0,129703,35353
16464,59,0,0,0,349621,12830
22386,30,2,2,2,972809,32800
10149,41,-2,-2,-2,24305,742
8729,43,0,0,0,53551,3648


In [12]:
#data_d = EqualWidthDiscretiser(bins=10, variables = ['AGE'])
#data_d.fit(data)
#data_t = data_d.transform(data)
#data_t.head(30)
#data = data_t

In [13]:
y = data['LIMIT_BAL']
print(y)

25665     40000
16464     80000
22386    170000
10149    200000
8729      50000
          ...  
4427     200000
948      470000
20532    180000
22839    120000
10093    210000
Name: LIMIT_BAL, Length: 1000, dtype: int64


In [14]:
# Split data into 80% train and 20% test. We will try also 70% and 30 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [15]:
#Algorithms


In [16]:
rf = RandomForestRegressor(random_state=SEED) 

In [17]:
from sklearn.model_selection import GridSearchCV
# Number of trees in random forest
#NOTE THAT I TRIED STOP =2000
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,3]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [5, 16, 28, 40, 51, 63, 75, 86, 98, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [2, 3], 'bootstrap': [True, False]}


In [18]:
n_folds = 5

In [19]:
clf = GridSearchCV(rf, random_grid, cv=n_folds, refit=False, scoring='r2', n_jobs=-1, verbose=1)

In [20]:
clf.fit(X_test, y_test)


Fitting 5 folds for each of 3960 candidates, totalling 19800 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=123), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [5, 16, 28, 40, 51, 63, 75, 86, 98, 110,
                                       None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [2, 3],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 14, 18, 23, 27, 32, 36, 41, 45,
                                          50]},
             refit=False, scoring='r2', verbose=1)

In [21]:
clf.best_params_

{'bootstrap': False,
 'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 23}

In [22]:
output_cv = pd.DataFrame(clf.cv_results_)

In [23]:
output_cv.to_csv("output_CV_rf.csv")

In [24]:
print(output_cv.head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.059605      0.010423         0.009796        0.006048   
1       0.080469      0.007736         0.005754        0.001469   
2       0.096737      0.006323         0.011236        0.005759   
3       0.113904      0.003601         0.011932        0.004784   
4       0.158881      0.015478         0.027117        0.009579   

  param_bootstrap param_max_depth param_max_features param_min_samples_leaf  \
0            True               5               auto                      2   
1            True               5               auto                      2   
2            True               5               auto                      2   
3            True               5               auto                      2   
4            True               5               auto                      2   

  param_min_samples_split param_n_estimators  \
0                       2                 10   
1                       2 